In [1]:
import pandas as pd

# Let's create dictionaries with required column names and data types 

# Dictionary sheet
dtype_dict = {'ID' : 'str',
              'Level' : 'str',
              'Level ID' : 'str'
             }

# NewData sheet
dtype_final = {'Level ID' : 'str',
               'ID' : 'str',
               'Code' : 'str',
               'Manager Number' : 'str',
               'Order' : 'int64'
              }

# Turning worsheets into DataFrames
new_current = pd.read_excel("The Manager.xlsm", sheet_name="NewCurrent")
dictionary = pd.read_excel("The Manager.xlsm", sheet_name="Dictionary", dtype=dtype_dict)

# The DataFrame below was created only to compare the final results
final_file = pd.read_excel("The Manager.xlsm", sheet_name="NewData", dtype=dtype_final)

In [2]:
# Let's see what the final result created with VBA looks like
final_file

,Level ID,Level Name,ID,Code,Name,Manager Number,Manager Name,Email,Order
0,10,Level 1,20,C8,Manchester,68331242,Penelope Cooper,False,1
1,10,Level 1,21,C9,London,91440508,Paige Morris,False,1
2,10,Level 1,2,C10,Newcastle,00539929,Adelaide Harper,False,1
3,10,Level 1,3,C11,Glasgow,64342267,Adrian Andrews,False,1
4,10,Level 1,19,C7,Paris,04157069,Reid Perry,False,1
...,...,...,...,...,...,...,...,...,...
125,16,Level 4,15,C3,Krakow,42979165,Michael Thompson,True,3
126,16,Level 4,9,C17,San Francisco,68918163,Heather Scott,True,3
127,16,Level 4,10,C18,San Diego,36603121,Justin Harris,True,3
128,16,Level 4,11,C19,Nashville,20671957,Stella Gray,False,3


In [3]:
# Let's copy the Dictionary & NewCurrent sheets
df_dict = dictionary.copy()
df_current = new_current.copy()

# Dropping the columns from Dictionary that have no headers
df_dict = df_dict[df_dict.columns.drop(list(df_dict.filter(regex='Unnamed')))]

# Creating necessary dictionaries from the Dictionary worksheet
code_id_dict = dict(zip(df_dict['Code'], df_dict['ID']))
level_match_dict = dict(zip(df_dict['Level Name Match'], df_dict['Level']))
level_code_dict = dict(zip(df_dict['Level ID'], df_dict['Level Name']))

# Two variables needed to create the Group column
level_number = 'Level 0'
group_counter = 0

# Let's find the City ID & City Name column positions
city_id = df_current.columns.get_loc('City ID')
city_name = df_current.columns.get_loc('City Name')

# And create an empty List for DataFrames
df_list = []

# Let's create a tuple with Level Names
levels = tuple(list(df_dict['Level Name Match'].dropna()))

In [4]:
# Let's see our tuple
levels

('Level 1 Manager',
 'Level 1 Assistant',
 'Level 1 Assistant 2',
 'Level 2 Manager',
 'Level 2 Assistant',
 'Level 2 Assistant 2',
 'Level 4 Manager',
 'Level 4 Assistant',
 'Level 4 Assistant 2')

In [5]:
# Let's work in a loop and go through each required Level
for lv in levels:
    # Let's find a position of required column
    level_column = df_current.columns.get_loc(lv)
    
    # Creating a new DataFrame (mydf) based on NewCurrent but with columns required only
    mydf = df_current.iloc[:,[city_id,city_name,level_column,level_column+1]].copy()
    
    # Drop all rows where there are no managers for required level
    mydf = mydf.dropna(subset=[lv])
    
    # If there are no managers for the level, go to the next iteration
    if len(mydf) == 0:
        continue
    
    # Rename columns 
    mydf.columns = ['Code', 'Name', 'Manager', 'Email']
    
    # Extract the manager's name from the Manager column
    mydf['Manager Name'] = mydf['Manager'].str.split('(').str[0].str.strip()
    
    # Extract the manager's number from the Manager column
    mydf['Manager Number'] = mydf['Manager'].str.split('(').str[1].str.strip(')')
    
    # Replace Yes with True and No with False in the Email column
    mydf['Email'] = mydf['Email'].str.lower() == 'yes'
    
    # Add ID based on the Code column and code_id_dict dictionary
    mydf['ID'] = mydf['Code'].map(code_id_dict)
    
    # Add Level ID column
    mydf['Level ID'] = lv
    
    # Add ID number based on Level ID and level_match_dict dictionary
    mydf['Level ID'] = mydf['Level ID'].map(level_match_dict)
    
    # Add Level Name based on Level ID and level_code_dict dictionary
    mydf['Level Name'] = mydf['Level ID'].map(level_code_dict)
    
    # Get the value of Level Name and assign it to order_check variable
    order_check = mydf['Level Name'].values[1]
    
    # If the level number hasn't changed, increase the number of group_counter by 1
    if order_check == level_number:
        group_counter += 1
        # And add this number to the Order column
        mydf['Order'] = group_counter
        # Update the level_number variable
        level_number = mydf['Level Name'].values[1]
        
    # If the level number has changed, set group_counter as 1
    else:
        group_counter = 1
        # And add this number to the Order column
        mydf['Order'] = group_counter
        level_number = mydf['Level Name'].values[1]
        # Update the level_number variable
    
    # Limit the DateFrame to the required columns in the correct order
    mydf = mydf[['Level ID', 'Level Name', 'ID', 'Code', 'Name', 'Manager Number', 'Manager Name', 'Email', 'Order']]
    
    # Add mydf DataFrame to the df_list
    df_list.append(mydf)
    
# Create final DataFrame by concatenating all the DataFrames from the df_list
full = pd.concat(df_list,ignore_index=True)

In [6]:
# Let's see how our result looks
full

,Level ID,Level Name,ID,Code,Name,Manager Number,Manager Name,Email,Order
0,10,Level 1,20,C8,Manchester,68331242,Penelope Cooper,False,1
1,10,Level 1,21,C9,London,91440508,Paige Morris,False,1
2,10,Level 1,2,C10,Newcastle,00539929,Adelaide Harper,False,1
3,10,Level 1,3,C11,Glasgow,64342267,Adrian Andrews,False,1
4,10,Level 1,19,C7,Paris,04157069,Reid Perry,False,1
...,...,...,...,...,...,...,...,...,...
125,16,Level 4,15,C3,Krakow,42979165,Michael Thompson,True,3
126,16,Level 4,9,C17,San Francisco,68918163,Heather Scott,True,3
127,16,Level 4,10,C18,San Diego,36603121,Justin Harris,True,3
128,16,Level 4,11,C19,Nashville,20671957,Stella Gray,False,3


In [7]:
# At first glance, our result is the same as the one obtained with VBA but let's make sure it's true.
full.equals(final_file)

True

In [8]:
# Finally, let's save our DataFrame to an Excel file in the results subfolder
full.to_excel("results/NewDataPandas.xlsx", sheet_name='NewData', index=False)